# RSNA Intracranial Aneurysm Detection - 3D CNN Pipeline

Este notebook implementa un pipeline de entrenamiento y predicción basado en una red neuronal convolucional 3D (3D CNN) usando PyTorch, para la detección y localización de aneurismas cerebrales en el dataset de la competición RSNA. Los datos se leen desde la carpeta `data/` y los resultados se guardan en `output/`.

In [1]:
# Sección 1: Importar librerías y definir rutas
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pydicom
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

DATA_DIR = '/kaggle/input/rsna-intracranial-aneurysm-detection'
SERIES_DIR = os.path.join(DATA_DIR, 'series')
TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
LOCALIZERS_CSV = os.path.join(DATA_DIR, 'train_localizers.csv')
OUTPUT_DIR = 'output'
os.makedirs(OUTPUT_DIR, exist_ok=True)
LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
]

In [3]:

# Sección 2: Cargar datos y fusionar localizadores (manteniendo todas las series)
train_df = pd.read_csv(TRAIN_CSV)
print('Shape train_df:', train_df.shape)
display(train_df.head())
localizers_df = pd.read_csv(LOCALIZERS_CSV)
print('Shape localizers_df:', localizers_df.shape)
display(localizers_df.head())
# Fusionar, manteniendo todas las series de train.csv y añadiendo info de localizadores si existe
df = pd.merge(train_df, localizers_df, on='SeriesInstanceUID', how='left', suffixes=('', '_localizer'))
print('Shape df fusionado:', df.shape)
display(df.head())

Shape train_df: (4405, 18)


,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
0,1.2.826.0.1.3680043.8.498.10004044428023505108...,64,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,76,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,1.2.826.0.1.3680043.8.498.10009383108068795488...,71,Male,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.8.498.10012790035410518400...,48,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Shape localizers_df: (2286, 4)


,SeriesInstanceUID,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,"{'x': 194.87253141831238, 'y': 178.32675044883...",Right Middle Cerebral Artery
2,1.2.826.0.1.3680043.8.498.10023411164590664678...,1.2.826.0.1.3680043.8.498.24186535344744886473...,"{'x': 189.23979878597123, 'y': 209.19184886465...",Right Middle Cerebral Artery
3,1.2.826.0.1.3680043.8.498.10030095840917973694...,1.2.826.0.1.3680043.8.498.75217084841854214544...,"{'x': 208.2805049088359, 'y': 229.78962131837307}",Right Infraclinoid Internal Carotid Artery
4,1.2.826.0.1.3680043.8.498.10034081836061566510...,1.2.826.0.1.3680043.8.498.71237104731452368587...,"{'x': 249.86745590416498, 'y': 220.623044646393}",Anterior Communicating Artery


Shape df fusionado: (4801, 21)


,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,...,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10004044428023505108...,64,Female,MRA,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,76,Female,MRA,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
3,1.2.826.0.1.3680043.8.498.10009383108068795488...,71,Male,MRA,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1.2.826.0.1.3680043.8.498.10012790035410518400...,48,Female,MRA,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
# Sección 3: Dataset y utilidades para 3D CNN
def load_dicom_volume(series_uid, target_shape=(64, 128, 128)):
    series_path = os.path.join(SERIES_DIR, str(series_uid))
    if not os.path.exists(series_path):
        return np.zeros(target_shape, dtype=np.float32)
    dicom_files = [f for f in os.listdir(series_path) if f.endswith('.dcm')]
    if not dicom_files:
        return np.zeros(target_shape, dtype=np.float32)
    dicom_files.sort()
    slices = []
    for f in dicom_files:
        dcm = pydicom.dcmread(os.path.join(series_path, f))
        img = dcm.pixel_array.astype(np.float32)
        # Forzar a 2D
        if img.ndim == 1:
            img = np.expand_dims(img, axis=0)
        elif img.ndim > 2:
            img = img.squeeze()
        if img.ndim != 2:
            continue  # descarta imágenes corruptas
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-6)
        slices.append(img)
    if len(slices) == 0:
        return np.zeros(target_shape, dtype=np.float32)
    try:
        volume = np.stack(slices, axis=0)
    except Exception:
        return np.zeros(target_shape, dtype=np.float32)
    if volume.ndim == 2:
        volume = np.expand_dims(volume, axis=0)
    if volume.ndim != 3:
        return np.zeros(target_shape, dtype=np.float32)
    from scipy.ndimage import zoom
    factors = [t/s for t, s in zip(target_shape, volume.shape)]
    volume = zoom(volume, factors, order=1)
    return volume

class Aneurysm3DDataset(Dataset):
    def __init__(self, df, label_cols=LABEL_COLS, augment=False):
        self.df = df
        self.label_cols = label_cols
        self.augment = augment
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        vol = load_dicom_volume(row['SeriesInstanceUID'])
        if self.augment and np.random.rand() < 0.5:
            vol = np.flip(vol, axis=2).copy()
        vol = np.expand_dims(vol, 0)
        label = row[self.label_cols].values.astype(np.float32)
        return torch.tensor(vol, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


In [5]:
# Sección 4: Definir modelo 3D CNN
class Simple3DCNN(nn.Module):
    def __init__(self, out_dim=13):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(1, 16, 3, padding=1), nn.BatchNorm3d(16), nn.ReLU(),
            nn.MaxPool3d(2),
            nn.Conv3d(16, 32, 3, padding=1), nn.BatchNorm3d(32), nn.ReLU(),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, 3, padding=1), nn.BatchNorm3d(64), nn.ReLU(),
            nn.AdaptiveAvgPool3d((2,4,4)),
        )
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*2*4*4, 128), nn.ReLU(),
            nn.Linear(128, out_dim),
        )
    def forward(self, x):
        x = self.net(x)
        x = self.head(x)
        return x

In [ ]:
# Sección 5: Entrenamiento de la 3D CNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_df = train_df.dropna(subset=LABEL_COLS, how='all').reset_index(drop=True)
train_idx, val_idx = train_test_split(np.arange(len(train_df)), test_size=0.2, random_state=42)
train_set = Aneurysm3DDataset(train_df.iloc[train_idx].reset_index(drop=True), augment=True)
val_set = Aneurysm3DDataset(train_df.iloc[val_idx].reset_index(drop=True), augment=False)
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=2, shuffle=False, num_workers=2)
model = Simple3DCNN(out_dim=len(LABEL_COLS)).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
best_auc = 0
for epoch in range(1, 11):
    print(f'Epoch {epoch}')
    model.train()
    train_losses = []
    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    model.eval()
    val_losses, preds, targets = [], [], []
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_losses.append(loss.item())
            preds.append(torch.sigmoid(out).cpu().numpy())
            targets.append(y.cpu().numpy())
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    aucs = []
    for i in range(len(LABEL_COLS)):
        try:
            auc = roc_auc_score(targets[:,i], preds[:,i])
        except:
            auc = np.nan
        aucs.append(auc)
    mean_auc = np.nanmean(aucs)
    print(f'Train loss: {np.mean(train_losses):.4f} | Val loss: {np.mean(val_losses):.4f} | Mean AUC: {mean_auc:.4f}')
    if mean_auc > best_auc:
        best_auc = mean_auc
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'best_3dcnn.pth'))
print('Entrenamiento finalizado. Mejor modelo guardado.')

Epoch 1


  0%|          | 3/1762 [00:30<4:34:23,  9.36s/it]

# Sección 6: Predicción con el modelo 3D CNN
Para predecir sobre nuevos datos, utiliza el script `scripts/predict_3dcnn.py` o adapta el siguiente ejemplo de código en una celda de este notebook.

In [ ]:
# Ejemplo de predicción sobre test.csv
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from scripts.train_3dcnn import Simple3DCNN, load_dicom_volume, LABEL_COLS
DATA_DIR = 'data'
SERIES_DIR = os.path.join(DATA_DIR, 'series')
TEST_CSV = os.path.join(DATA_DIR, 'test.csv')
OUTPUT_DIR = 'output'
class Aneurysm3DTestDataset(Dataset):
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        vol = load_dicom_volume(row['SeriesInstanceUID'])
        if vol is None:
            vol = np.zeros((64,128,128), dtype=np.float32)
        vol = np.expand_dims(vol, 0)
        return torch.tensor(vol, dtype=torch.float32), row['SeriesInstanceUID']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_df = pd.read_csv(TEST_CSV)
test_set = Aneurysm3DTestDataset(test_df)
test_loader = DataLoader(test_set, batch_size=2, shuffle=False, num_workers=2)
model = Simple3DCNN(out_dim=len(LABEL_COLS)).to(device)
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'best_3dcnn.pth'), map_location=device))
model.eval()
results = []
with torch.no_grad():
    for x, series_uids in test_loader:
        x = x.to(device)
        out = torch.sigmoid(model(x)).cpu().numpy()
        for i, uid in enumerate(series_uids):
            row = {'SeriesInstanceUID': uid}
            for j, col in enumerate(LABEL_COLS):
                row[col] = out[i, j]
            results.append(row)
pred_df = pd.DataFrame(results)
pred_df.to_csv(os.path.join(OUTPUT_DIR, '3dcnn_predictions.csv'), index=False)
print('Predicciones guardadas en', os.path.join(OUTPUT_DIR, '3dcnn_predictions.csv'))

# Fin del pipeline 3D CNN
Todos los modelos y el resumen de resultados se han guardado en `/kaggle/working` y pueden descargarse desde la interfaz de Kaggle tras la ejecución del notebook.